# CT5170: Principles of ML - Assignment 2
## Course code: 1MAO3
### Participants (name: id): (Daniel Verdejo: 22240224, Thomas Cronin: 22239435)

# Report

As we are doing this assignment in a pair we chose to split the work based on complexity:
(Data Exploration/Clean-up, metric creations, Perceptron)'s work load is equal to the Multi Layer Perceptron work load.
As such we decided that Tom would do the Perceptron, Data and metrics and Danny would do the Multi Layer Perceptron
* Note: The metrics is the creation of the scoring algorithm, not the metrics for an algorithm as such we calculate our own metrics for our own algorithm using the helper metrics

### Data Exploration and Clean-up
##### Tom's Work
A helper function was created to load the dataset into a pandas DataFrame; this function ‘read_data_return_dataframe’ takes a string containing the path to a txt file. It reads the dataset via pandas.read() and returns the DataFrame containing the data.

Exploring the data showed the ranges and statistical data, while being quite distinct from each other and all in a usable state, would require normalisation. The ‘fire’ column, however, did not look correct; it contained eight unique entries for what should have been two individual entries, ‘yes’ or ‘no’. It seemed like the data was being read with questionable spaces giving them, as far as panda DataFrames are concerned, unique entries. To replace the entries, a convert_label function was implemented to replace a specified binary value with a different binary value. This allowed the change from ‘yes/no’ to ‘1/0’ while also replacing duplicate entries.

The dataset is imbalanced, slightly skewed to the positive case ‘yes’. There are 97 ‘no’ and 107 ‘yes’.

To normalise the data, the helper function Normalize was implemented to normalise all the data to 0-1 ranges. It takes a DataFrame and a list of features, returning a DataFrame of the normalised value of each entry in each feature column.
### Metrics
#### Tom's Work
### Algorithms
#### Perceptron Algorithm
##### Tom's Work
<u>Design Decisions:</u>
* I choose not to include a bias modification for the perceptron as it would not be needed to hopefully get a high accuracy on this dataset, based on the data exploration.

* This Perceptron will be needed to both classify in binary, as-well as non-binary outputs, so it is possible to specify one of three activations functions as a parameter ('ReLU', 'heaviside' and 'sigmoid') and the perceptron will use its own implementation of the specified activation function.

* The perceptron initiates the weights if not specified between random values of (-0.5, 0.5) after researching the algorithm this was deemed a suitable weight range. It is also possible to pass predefined weights to the nueron.

* For the sigmoid activation function it was decided that a summed weights=0, which in sigmoid would return a 0.5 would still be a positive case and would activate the perceptron with a 1.

<u>Algorithm Design</u>
*   Pseudo Code:

        ThresholdLogicUnit()

            Learning Rate = Input()
            Input Weights = Initialised Randomly or Input
            Activation Functions = Heaviside or Input

            fit()
                Training Set = Input()
                Labels = Input() or None
                Learning Iterations = Input or 200

                for each iteration:
                    predict value on inputs
                    compare predictions with results
                    if prediction is wrong update weights:
                        new weights = weighs -  value of learning rate multiplied by (prediction - label) multiplied by input

            predict()
                samples = Input
                for each sample predict a score
                return a list of the predictions

* Description:

    The Perceptron Algorithm:
        The Threshold Logic Unit takes in n_dimentional data and there corresponding 1_dimentional array of labels. It requires a learning rate, which is used to tune the changing of the weights values. After getting the data and labels, the weights for the amount of features in a sample (the input shape) is initialised  randomly between values of -0.5,+0.5 (note: these can be set aswell). The Threshold Logic Unit then loops for the amount of iterations it should run and trains on each sample. During the training process it will predict a score using the specified activation function. If the score is not equal to the coresponding label, the weights are updated by subtracting the current weights with the learning rate * error * the input. Once the TLU has completed its training, we can then pass it an unseen sample and get a prediction.
#### Multi Layer Perceptron Algorithm
##### Danny's Work

### Test's and Result's
#### Perceptron
##### Tom's Work
#### Multi Layer Perceptron
### Conclusion and observations
##### Tom and Danny's Conclusions and observaitons



# Implementation

In [ ]:
import pandas as pd
import numpy as np
from seaborn import pairplot
from Utils import *
from Metrics import *

from ThresholdLogicUnit import ThresholdLogicUnit

from sklearn.linear_model import Perceptron
from sklearn.model_selection import train_test_split


### Data Exploration

In [ ]:
# Tom Cronin
loaded_data_frame = read_data_return_dataframe("../wildfires.txt") # Loads The wildfire Dataset
wildfires_df = loaded_data_frame.copy() # copys the data so we don't mess with the original dataset

print(wildfires_df.shape) # gets the dimensions of the dataframe
print("-" * 20)
print(wildfires_df.columns) # gets the features of the columns
print("-" * 20)
print(wildfires_df.dtypes) # returns the datatypes
print("-" * 20)
print(wildfires_df.describe(include='all'))

#### Error in fire label?
It seems that the fire label is a mix of 'no' and 'yes' however there are different spaces causing the dataframe to think there are multiple entries, 8 instead of 2. Let's fix that

Fire is now a binary class

### Perceptron Algorithm
For sake of clarity and shortness of this file, the perceptron was implemented as a ThresholdLogicUnit in a seperate file. Can be found in the zip

In [2]:
from MLP import MLP
from Utils import *
from sklearn.neural_network import MLPClassifier
from Metrics import *
import random

random.seed(2060)

data = read_data_return_dataframe("../wildfires.txt")
# Copy to be used for the rest of the assignment
wildfires_copy = data.copy()
# wildfires_copy = convert_label(wildfires,'fire',['no', 'yes'],[0, 1])


features = ['year', 'temp', 'humidity', 'rainfall', 'drought_code', 'buildup_index', 'day', 'month', 'wind_speed']
X_train, X_test, y_train, y_test = split_df_to_train_test_dfs(wildfires_copy, test_set_size=.2,
                                                    random_state=42)
X_train = X_train[features].values  # returns a numpy NdArray of the features
X_test = X_test[features].values  # returns a numpy NdArray of the features
X_train = Normalize(X_train, features)
X_test = Normalize(X_test, features)

X_train = np.asarray(X_train)[0:40]
y_train = np.asarray(y_train).flatten()
y_test = np.asarray(y_test).flatten()

y_train = np.asarray([1 if 'yes' in y else 0 for y in y_train])[0:40]
X_test = np.asarray(X_test)[0:40]
y_test = np.asarray([1 if 'yes' in y else 0 for y in y_test])[0:40]

m, n = X_train.shape
mlp = MLP()
mlp.add_layer(output_size = m, activation='relu', input_size=n) # Add a layer of 9 inputs and 32 outputs
mlp.add_layer(output_size = 1, activation='sigmoid', input_size= m) # add output layer with 32 inputs and 1 output
mlp.train(X=X_train, Y=y_train, iters=200, show_metrics=True)

clf = MLPClassifier(random_state=1, max_iter=200).fit(X_train, y_train)
p = clf.predict(X_test)

z = mlp.predict(X_test)[0]
print(f"sklearn:MLPClassifier: {p}, actual: {y_test}")
print(f"false and true positives, false & true negatives, {confusion_matrix(p, y_test)}")
print(f"accuracy: {accuracy(p, y_test)}")
print(f"recall: {recall(p, y_test)}")
print(f"f1_score: {f1_score(p, y_test)}")
print(f"precision: {precision(p, y_test)}")

print(f"MLP(): {z}, actual: {y_test}")
print(f"false and true positives, false & true negatives, {confusion_matrix(z, y_test)}")
print(f"accuracy: {accuracy(z, y_test)}")
print(f"recall: {recall(z, y_test)}")
print(f"f1_score: {f1_score(z, y_test)}")
print(f"precision: {precision(z, y_test)}")



iteration: 0 / 200 ============= Acc: 0.5
iteration: 25 / 200 ============= Acc: 0.5
iteration: 50 / 200 ============= Acc: 0.5
iteration: 75 / 200 ============= Acc: 0.5
iteration: 100 / 200 ============= Acc: 0.5
iteration: 125 / 200 ============= Acc: 0.5
iteration: 150 / 200 ============= Acc: 0.5
iteration: 175 / 200 ============= Acc: 0.5
Prediction: [[1 1 0 0 0 1 1 0 1 0 1 1 1 1 1 1 1 1 0 0 1 1 1 1 1 0 1 0 0 1 1 0 1 0 1 1
  1 1 0 0]]
sklearn:MLPClassifier: [1 0 1 1 0 0 0 0 0 0 1 1 0 1 0 0 1 0 0 0 0 1 0 0 1 0 0 0 0 1 0 1 1 0 1 0 1
 0 1 0], actual: [1 0 1 1 0 0 1 1 0 0 1 1 0 1 0 1 1 0 1 0 0 1 1 0 1 1 0 1 1 1 0 1 0 1 1 0 1
 0 1 0]
false and true positives, false & true negatives, [[14, 1], [9, 16]]
accuracy: 0.75
recall: 0.6086956521739131
f1_score: 0.7368421052631579
precision: 0.9333333333333333
MLP(): [1 1 0 0 1 1 0 1 0 1 1 1 0 1 1 0 1 1 1 1 1 1 1 0 1 1 1 1 0 1 1 1 0 1 1 1 1
 1 1 1], actual: [1 0 1 1 0 0 1 1 0 0 1 1 0 1 0 1 1 0 1 0 0 1 1 0 1 1 0 1 1 1 0 1 0 1 1 0 1
 0 1 0]
false

c:\Users\danny\anaconda3\envs\tensorflow\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


## MultiLayer Peceptron

For the comparrison of our implementation of the MultiLayer Perceptron the decision was made to use MLPClassifier found in sklearn.neural_network package. It can be seen from the scores above that sklearns implementation is better at classifying the data over our implementation of the MultiLayer Perceptron(MLP). The reasoning behind this is likely due to some design decisions which we're made on the perceptron, which will be discussed shortly.

First lets review the design of the MLP neural network(NN) model.

The MLP is built up of `n` amount of layers. When we construct the object we initialise its layers and weights to empty lists. From there, every time the `add_layer` function is called, the weights are added for the given layer. A layer consists of 'm' number of neurons, our Threshold Logic Unit(TLU) perceptrons. The layer itself manages its neurons with respect to how its `fit, predict, update, and initialise` functions are called. The layer passes arguments down to each of its child neurons including the `learning_rate` and `activation`. The activation function defines what the neurons outputs will be, whether they are `0` and `1` as you would get from a sigmoid or heaviside function, or a value ranging from -.5 to greater than 0 that we get from the ReLU activation function.

Reverting to the MLP NN, some functions are defined on the class as: forward_propegation, back_propegation, train, and predict.

1. The `forward_propegation` function is the process of forwarding the output of the previous hidden layer into the layer which follows. The input -> first hidden layer is unique as it receives `X`, our dataset in its "raw" form (after normalisation, splitting and so on.). The first hidden layer in our example is initialised using the ReLU function for its child neurons. The output of the layer is propegated to the next layer in the network, which will pass its output to the layer which follows it until we come to the output layer, the end of the NN. The output layer returns a list of classifications that is the size of `n` number of samples (e.g. 50 samples are fed to the NN, 50 classifications are returned from the NN).
2. The `back_propegation` function is the process of feeding back the error and delta in our outputs relative the the labels for every layer in the network. The error is gathered by comparing the output `Z` of the `forward_propegation` function relative to the labels `Y`. The delta is then calculated by multiplying the error of the output by the derivative of sigmoid `Z`.  With this information the NN can update the weights for each layer based off the dot products of the individual error and delta for each layer. The update value from the dot product of the transposed `X`, `d` (delta per layer) is passed to the layer which feeds the update into its neurons (perceptrons). Typically the bias would also be calculated, but the decision was made to exclude the bias from the perceptron, additionally removing the need to add or process biases in the `forward_propegation` and `back_propegate` functions.
3. The `train` function simply calls the `forward_propegation` and `back_propegation` functions `n` times. It also has the ability to show metrics as the iterations go by. Finally it returns the outputs of the `forward_propegation` function.
4. The `predict` function similar to the `train` function calls `forward_propegation` but only once and returns its output.

Overall the NN's performance shows ~20 - 30% less accurate when compared to sklearns implementation. Additionally the recall, f1 score and precision show values of ~10 - 30 points lower on average when compared against sklearns implementation. Conclusion more time and work is needed to improve the MLP NN.

As briefly mentioned in the `back_propegation` function description, when comparing the local MLP implementation against the third party implementation, it is a possibility that while not including biases on a perceptron level has a negligable effect, it is likely that the omission on a network level could lead to poorer performance. _While this decision simplified the implementation on all levels (Perceptron, Layer, MLP), some more time and experimentation with these biases added in would show whether the omission of the biases was the right choice_. 

Splitting data into training_data, testing_data, training_labels, testing_labels is done using some simple dataframe functionality. First by splitting the dataset into 2 fractions (train and test) typically around 80 - 90% train with the remainder being test. These are then further split down into their labels and features / attributes using  `df.iloc` with some simple list comprehension taking all rows and segregating by column Finally to normalise the labels another simple list comprehension is used to map values 1 to every 'yes' found, and 0 to every 'no' found in the labels.

Some additional footnotes: To enhance the robustness of the classes and functions found throughout the application both unit tests and validators are added. These also describe the functionality of the the classes and functions allowing for easier understanding when working in a team and reduces the likelyhood of user error when using said classes and functions.